In [6]:
import itertools

import yaml
import pandas as pd
import numpy as np
import sklearn.metrics
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

import lab.metrics
import lab.analytics

In [7]:
def load_results():
    combinations = [
        (classifier, defence, mode, suffix)
        for classifier in ["kfp", ]
        for defence in ["tamaraw", ]
        for mode in ["drop", "delay", "delay.10chaff", "delay.3kmsd", "delay.sizes", "delay.300mult"]
        for suffix in ["", ".sim"]
        if not (mode in ("delay.10chaff", "delay.3kmsd", "delay.sizes") and suffix == ".sim")
    ]
        
    return (pd.concat([
        pd.read_csv(f"../../results/delay-vs-drop/{defence}/{mode}/predict{suffix}/{classifier}-0.csv", header=0)
        for (classifier, defence, mode, suffix) in combinations
    ], keys=combinations, names=["classifier", "defence", "mode", "suffix"])
            .fillna(0)
            .rename(columns=lambda c: int(c) if c != "y_true" else c)
           )#.drop(columns=[-1]))


data = load_results()
data

y_true         0    1    2    3  \
classifier defence mode          suffix                                        
kfp        tamaraw drop                 0       0.0  0.833333  0.0  0.0  0.0   
                                        1       0.0  1.000000  0.0  0.0  0.0   
                                        2       0.0  0.500000  0.0  0.0  0.0   
                                        3       0.0  0.000000  0.0  0.0  0.0   
                                        4       0.0  1.000000  0.0  0.0  0.0   
...                                             ...       ...  ...  ...  ...   
                   delay.300mult .sim   395    19.0  0.000000  0.0  0.0  0.0   
                                        396    19.0  0.000000  0.0  0.0  0.0   
                                        397    19.0  0.000000  0.0  0.0  0.0   
                                        398    19.0  0.000000  0.0  0.0  0.0   
                                        399    19.0  0.000000  0.0  0.0  0.0   

                                                    4         5    6    7  \
classifier defence mode          suffix                                     
kfp        tamaraw drop                 0    0.000000  0.166667  0.0  0.0   
                                        1    0.000000  0.000000  0.0  0.0   
                                        2    0.000000  0.500000  0.0  0.0   
                                        3    0.000000  1.000000  0.0  0.0   
                                        4    0.000000  0.000000  0.0  0.0   
...                                               ...       ...  ...  ...   
                   delay.300mult .sim   395  0.666667  0.000000  0.0  0.0   
                                        396  0.666667  0.000000  0.0  0.0   
                                        397  0.666667  0.000000  0.0  0.0   
                                        398  0.666667  0.000000  0.0  0.0   
                                        399  0.666667  0.000000  0.0  0.0   

                                               8  ...   10   11   12   13  \
classifier defence mode          suffix           ...                       
kfp        tamaraw drop                 0    0.0  ...  0.0  0.0  0.0  0.0   
                                        1    0.0  ...  0.0  0.0  0.0  0.0   
                                        2    0.0  ...  0.0  0.0  0.0  0.0   
                                        3    0.0  ...  0.0  0.0  0.0  0.0   
                                        4    0.0  ...  0.0  0.0  0.0  0.0   
...                                          ...  ...  ...  ...  ...  ...   
                   delay.300mult .sim   395  0.0  ...  0.0  0.0  0.0  0.0   
                                        396  0.0  ...  0.0  0.0  0.0  0.0   
                                        397  0.0  ...  0.0  0.0  0.0  0.0   
                                        398  0.0  ...  0.0  0.0  0.0  0.0   
                                        399  0.0  ...  0.0  0.0  0.0  0.0   

                                              14        15   16   17  \
classifier defence mode          suffix                                
kfp        tamaraw drop                 0    0.0  0.000000  0.0  0.0   
                                        1    0.0  0.000000  0.0  0.0   
                                        2    0.0  0.000000  0.0  0.0   
                                        3    0.0  0.000000  0.0  0.0   
                                        4    0.0  0.000000  0.0  0.0   
...                                          ...       ...  ...  ...   
                   delay.300mult .sim   395  0.0  0.166667  0.0  0.0   
                                        396  0.0  0.166667  0.0  0.0   
                                        397  0.0  0.166667  0.0  0.0   
                                        398  0.0  0.166667  0.0  0.0   
                                        399  0.0  0.166667  0.0  0.0   

                                                   18   19  
cl

In [8]:
data["y_pred"] = data.drop(columns="y_true").idxmax(axis=1)
data

y_true         0    1    2    3  \
classifier defence mode          suffix                                        
kfp        tamaraw drop                 0       0.0  0.833333  0.0  0.0  0.0   
                                        1       0.0  1.000000  0.0  0.0  0.0   
                                        2       0.0  0.500000  0.0  0.0  0.0   
                                        3       0.0  0.000000  0.0  0.0  0.0   
                                        4       0.0  1.000000  0.0  0.0  0.0   
...                                             ...       ...  ...  ...  ...   
                   delay.300mult .sim   395    19.0  0.000000  0.0  0.0  0.0   
                                        396    19.0  0.000000  0.0  0.0  0.0   
                                        397    19.0  0.000000  0.0  0.0  0.0   
                                        398    19.0  0.000000  0.0  0.0  0.0   
                                        399    19.0  0.000000  0.0  0.0  0.0   

                                                    4         5    6    7  \
classifier defence mode          suffix                                     
kfp        tamaraw drop                 0    0.000000  0.166667  0.0  0.0   
                                        1    0.000000  0.000000  0.0  0.0   
                                        2    0.000000  0.500000  0.0  0.0   
                                        3    0.000000  1.000000  0.0  0.0   
                                        4    0.000000  0.000000  0.0  0.0   
...                                               ...       ...  ...  ...   
                   delay.300mult .sim   395  0.666667  0.000000  0.0  0.0   
                                        396  0.666667  0.000000  0.0  0.0   
                                        397  0.666667  0.000000  0.0  0.0   
                                        398  0.666667  0.000000  0.0  0.0   
                                        399  0.666667  0.000000  0.0  0.0   

                                               8  ...   11   12   13   14  \
classifier defence mode          suffix           ...                       
kfp        tamaraw drop                 0    0.0  ...  0.0  0.0  0.0  0.0   
                                        1    0.0  ...  0.0  0.0  0.0  0.0   
                                        2    0.0  ...  0.0  0.0  0.0  0.0   
                                        3    0.0  ...  0.0  0.0  0.0  0.0   
                                        4    0.0  ...  0.0  0.0  0.0  0.0   
...                                          ...  ...  ...  ...  ...  ...   
                   delay.300mult .sim   395  0.0  ...  0.0  0.0  0.0  0.0   
                                        396  0.0  ...  0.0  0.0  0.0  0.0   
                                        397  0.0  ...  0.0  0.0  0.0  0.0   
                                        398  0.0  ...  0.0  0.0  0.0  0.0   
                                        399  0.0  ...  0.0  0.0  0.0  0.0   

                                                   15   16   17        18  \
classifier defence mode          suffix                                     
kfp        tamaraw drop                 0    0.000000  0.0  0.0  0.000000   
                                        1    0.000000  0.0  0.0  0.000000   
                                        2    0.000000  0.0  0.0  0.000000   
                                        3    0.000000  0.0  0.0  0.000000   
                                        4    0.000000  0.0  0.0  0.000000   
...                                               ...  ...  ...       ...   
                   delay.300mult .sim   395  0.166667  0.0  0.0  0.166667   
                                        396  0.166667  0.0  0.0  0.166667   
                                        397  0.166667  0.0  0.0  0.166667   
                                        398  0.166667  0.0  0.0  0.166667   
                                        399  0.166667  0.0  0.0  0.166667   

In [9]:
accuracy = data.groupby(["classifier", "defence", "mode", "suffix"]).apply(lambda x: sklearn.metrics.accuracy_score(x["y_true"], x["y_pred"]) )
accuracy

classifier  defence  mode           suffix
kfp         tamaraw  delay                    0.8925
                                    .sim      0.1775
                     delay.10chaff            0.9425
                     delay.300mult            0.9025
                                    .sim      0.1300
                     delay.3kmsd              0.9300
                     delay.sizes              0.8700
                     drop                     0.9475
                                    .sim      0.1450
dtype: float64